# New method using TF-IDF vectorization

In [1]:
import numpy as np
from nltk import word_tokenize
from pip._internal.utils.misc import tabulate
from sklearn.feature_extraction.text import TfidfVectorizer
import glob
import pandas as pd
%run stylometric_Features.ipynb import FeatureExtration
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib as plt
import warnings
import string
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC
from nltk.stem import WordNetLemmatizer
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
%matplotlib


Using matplotlib backend: Qt5Agg


In [2]:
all_instances={

'hrudayashiva':["01","02","03","04","05","06","07","08","09",10,11,12],
    'ravibeliger':[13,14,15,16,17,18,19,20,21,22,23,24,25],
    'somashaker':[26,27,28,29,30,31,32,33,34,35,36],
    'Other author':[]
}
#from features_NormalApproach import FeatureExtration


stop_words_list = "C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/new_stop_words.txt"
data_folder = "C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/train"
instance_by_author={}
for author,AllFiles_Per_author in all_instances.items():
    for i in AllFiles_Per_author:
        for name in glob.glob(f"C://Users/RAVIKUMAR/PycharmProjects/Authorship_Attribution_Instance/Data_2/train/instance{i}.txt"):
            with open(name, encoding='utf-8') as file:
                instance_by_author[author+str(i)]=file.read()

In [3]:
lemmatiser = WordNetLemmatizer()
stopword=[]
with open(stop_words_list, encoding='utf-8') as file:
    reader=file.read()
    stopword.append(reader)

def text_process(tex):
    nopunct = [char for char in tex if char not in string.punctuation]
    nopunct = ''.join(nopunct)
    a = ''
    for i in range(len(nopunct.split())):
        b = lemmatiser.lemmatize(nopunct.split()[i], pos="v")
        a = a + b + ' '
    return [word for word in a.split() if not word in stopword]
all_authors=[]
text=[]
for author, file in instance_by_author.items():
    all_authors.append(author)
    text.append(file)
    text_process(file)


df=pd.DataFrame()
df['author']=all_authors
df['author'][0:13]='hrudayashiva'
df['author'][13:25]='ravibeligeri'
df['author'][25:36]='somashaker'
df['text']=text
            

In [4]:
# for author, file in instance_by_author.items():
#     df['Sylometric']=pd.Series(FeatureExtration(file))

from sklearn.preprocessing import LabelEncoder
y = df['author']
# labelencoder = LabelEncoder()
# y = labelencoder.fit_transform(y)
X = df['text']
df.index=df.index+1
from sklearn.model_selection import train_test_split
# 80-20 splitting the dataset (80%->Training and 20%->Validation)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=1234)


In [7]:
#defining the bag-of-words transformer on the text-processed corpus # i.e., text_process() declared in II is executed...
#bow_transformer=CountVectorizer(ngram_range=(2,3),min_df=5, analyzer=text_process).fit(X_train)
word_vector=TfidfVectorizer(analyzer="word", ngram_range=(2,3),max_features=2000, binary=False).fit(X_train)
word_vector1=TfidfVectorizer(analyzer="word", ngram_range=(3,4),max_features=2000, binary=False).fit(X_train)




char_vector=TfidfVectorizer(analyzer="char", ngram_range=(2,3),max_features=2000, binary=False,min_df=3).fit(X_train)
char_vector1=TfidfVectorizer(analyzer="char", ngram_range=(3,3),max_features=2000, binary=False,min_df=3).fit(X_train)
bow_transformer=FeatureUnion([ ("chars",char_vector),("word",word_vector),("3_chars",char_vector1),("3_word",word_vector1)] )


#bow_transformer =  TfidfVectorizer(min_df=3, analyzer=text_process).fit(X_train)

#bow_transformer=FeatureExtration(X_train)
#transforming into Bag-of-Words and hence textual data to numeric..
feature_names = bow_transformer.get_feature_names()
text_bow_train=bow_transformer.transform(X_train)#ONLY TRAINING DATA

# transforming into Bag-of-Words and hence textual data to numeric..
text_bow_test=bow_transformer.transform(X_test)#TEST DATA


#print(bow_transformer.get_feature_names())

In [11]:
#print("num of features: ", (bow_transformer.get_feature_names()))
text_bow_train

<28x8000 sparse matrix of type '<class 'numpy.float64'>'
	with 81340 stored elements in Compressed Sparse Row format>

In [46]:
# Importing necessary libraries
# instantiating the model with Multinomial Naive Bayes..
model = SVC(kernel='linear')

# training the model...
model = model.fit(text_bow_train, y_train)
#import
print("score of the model :",model.score(text_bow_train, y_train))
print("testig score of the model",model.score(text_bow_test, y_test))

# Importing necessary libraries
from sklearn.metrics import classification_report

# getting the predictions of the Validation Set...

predictions = model.predict(text_bow_test)
pred = pd.DataFrame()
pred["text"] = X_test
pred["author"]=predictions
print(pred)
df['text']

score of the model : 1.0
testig score of the model 0.875
                                                 text        author
8   \nಅದೊಂದು ಮಾಮೂಲಿ ದಿನ… ಆ ಸಂಜೆ ಒಂದು ಕಾರ್ಯಕ್ರಮಕ್ಕೆ...  hrudayashiva
35  \n " ಇರುವುದೆಲ್ಲವ ಬಿಟ್ಟು ಇರದುದರೆಡೆಗೆ ತುಡಿವುದೇ ಜ...    somashaker
13  ಇತ್ತೀಚೆಗೊಂದು ಮಧ್ಯಾಹ್ನ ದಾವಣಗೆರೆಯಿಂದ ರವೀಂದ್ರನಾಥ್...  ravibeligeri
1   ಪ್ರಖ್ಯಾತ ಹಿಂದೂಸ್ತಾನಿ ಸಂಗೀತಗಾರ ಪಂಡಿತ್ ಪರಮೇಶ್ವರ ...  ravibeligeri
4   \n'ಈ ಹಾಡನ್ನು ಬೇರೆ ಯಾರಿಂದಲಾದರೂ ಬರೆಸಿಬಿಡಿ ಪ್ಲೀಸ್...  hrudayashiva
31  \nಪ್ರಪಂಚ ರಹಸ್ಯಗಳ, ವಿಸ್ಮಯಗಳ ಆಗರ! ಎಲ್ಲಾ ಜೀವಿಗಳಂತ...    somashaker
14  \n“ಅಬ್ಬ! ಎಂ.ಎ.? ಎಷ್ಟು ಕೊಡ್ತಾರಲೆ?" ಕಣ್ಣರಳಿಸಿ ಕೇ...  ravibeligeri
27  \nಮಾಂಗಲ್ಯಂ ತಂತುನಾನೇನ ಮಮಜೀವನ ಹೇತುನಾ … ಎಂಬ ಮಂತ್ರ...    somashaker


1     \nಹೋಳಿ ಹಬ್ಬದ ಸಂಭ್ರಮ, ಖುಷಿಗಳನ್ನು ನೀವೂ ಅನುಭವಿಸಿರ...
2     ಪ್ರಖ್ಯಾತ ಹಿಂದೂಸ್ತಾನಿ ಸಂಗೀತಗಾರ ಪಂಡಿತ್ ಪರಮೇಶ್ವರ ...
3     \nಟಾಮ್ ಹಾಂಕ್ಸ್ ಕ್ಯಾಲಿಫೋರ್ನಿಯಾದ ಪ್ರತಿಭಾವಂತ. ಈತ ...
4     \nಹೆಂಡತಿಯು ಹದ ಮಾಡಿಕೊಟ್ಟ ತಾಂಬೂಲ ಜಗಿಯುತ್ತಲೇ ಅವರು...
5     \n'ಈ ಹಾಡನ್ನು ಬೇರೆ ಯಾರಿಂದಲಾದರೂ ಬರೆಸಿಬಿಡಿ ಪ್ಲೀಸ್...
6     \nನಾನು ಹದಿನಾರನೇ ವಯಸ್ಸಿಗೆ ಹಳ್ಳಿಗಳನ್ನು ಬಿಟ್ಟಿದ್ದ...
7     \nದೆವ್ವಗಳ ಬಗ್ಗೆ ಚರ್ಚಿಸುವುದೇ ವೇಳೆ ಹಾಳುಮಾಡಿಕೊಳ್ಳ...
8     \nಅನಂತಮೂರ್ತಿಯವರ ಕುರಿತು ಒಂದಿಷ್ಟು ಧ್ಯಾನಿಸುವ ಸಮಯವ...
9     \nಅದೊಂದು ಮಾಮೂಲಿ ದಿನ… ಆ ಸಂಜೆ ಒಂದು ಕಾರ್ಯಕ್ರಮಕ್ಕೆ...
10    \nದಟ್ಟ ಮೌನ ಕವಿದಿದೆ. ತಾತನ ಗೋರಿಯ ಪಕ್ಕದಲ್ಲಿ ಕುಳಿತ...
11    \nಕಗ್ಗಲಿಪುರದಲ್ಲಿ 'ಕರುನಾಡ ಗಜಕೇಸರಿ ಸೇನೆ'ಯ ಉದ್ಘಾಟ...
12    \n\nಅದೇಕೋ ನೆನಪಾಗುತ್ತಿದೆ. ಐದಾರು ವರ್ಷಗಳ ಹಿಂದೆ ನಾ...
13    \nಆಕೆ ಟಿಕೆಟ್ ಕೌಂಟರ್‌ನ ಮುಂದೆ ನಿಂತು ಸಿನೆಮಾ ಚೆನ್ನ...
14    ಇತ್ತೀಚೆಗೊಂದು ಮಧ್ಯಾಹ್ನ ದಾವಣಗೆರೆಯಿಂದ ರವೀಂದ್ರನಾಥ್...
15    \n“ಅಬ್ಬ! ಎಂ.ಎ.? ಎಷ್ಟು ಕೊಡ್ತಾರಲೆ?" ಕಣ್ಣರಳಿಸಿ ಕೇ...
16    \nಇವತ್ತು ಮಂಜು, ಸುಮಿತ್ರ, ಶಶಿಧರ ಎಲ್ಲರೂ ಬೆಂಗಳೂರಿನ...
17    \nಗೆಳೆಯರು ಅವನನ್ನೇ ಆಶ್ಚರ್ಯದಿಂದ ನೋಡಿದರು. ಟೇಬಲ್‌ನ...
18    \nಈ ಹುಡುಗ ಸುದೀಪ್.\n\nಆತನ ಬಗ್ಗೆ ನನಗೊಂದು ಪ್ರ

In [14]:
 print("test data\n ",X_test,"\n")
# getting the Precision, Recall, F1-Score
print(classification_report(y_test, predictions))
#print(predictions,X_test)
with open("NewFile.txt",encoding="utf-8") as unknown:
    unknownFile=unknown.read()
unknownFile2=input("Enter the string\n")
df["NewData"]=unknownFile2
z_test=df["NewData"]
text_bow_test2=bow_transformer.transform(z_test)#TEST DATA

predictions2=model.predict(text_bow_test2)
s = set(predictions2)
print(s)

test data
  8     \nಅದೊಂದು ಮಾಮೂಲಿ ದಿನ… ಆ ಸಂಜೆ ಒಂದು ಕಾರ್ಯಕ್ರಮಕ್ಕೆ...
35    \n " ಇರುವುದೆಲ್ಲವ ಬಿಟ್ಟು ಇರದುದರೆಡೆಗೆ ತುಡಿವುದೇ ಜ...
13    ಇತ್ತೀಚೆಗೊಂದು ಮಧ್ಯಾಹ್ನ ದಾವಣಗೆರೆಯಿಂದ ರವೀಂದ್ರನಾಥ್...
1     ಪ್ರಖ್ಯಾತ ಹಿಂದೂಸ್ತಾನಿ ಸಂಗೀತಗಾರ ಪಂಡಿತ್ ಪರಮೇಶ್ವರ ...
4     \n'ಈ ಹಾಡನ್ನು ಬೇರೆ ಯಾರಿಂದಲಾದರೂ ಬರೆಸಿಬಿಡಿ ಪ್ಲೀಸ್...
31    \nಪ್ರಪಂಚ ರಹಸ್ಯಗಳ, ವಿಸ್ಮಯಗಳ ಆಗರ! ಎಲ್ಲಾ ಜೀವಿಗಳಂತ...
14    \n“ಅಬ್ಬ! ಎಂ.ಎ.? ಎಷ್ಟು ಕೊಡ್ತಾರಲೆ?" ಕಣ್ಣರಳಿಸಿ ಕೇ...
27    \nಮಾಂಗಲ್ಯಂ ತಂತುನಾನೇನ ಮಮಜೀವನ ಹೇತುನಾ … ಎಂಬ ಮಂತ್ರ...
Name: text, dtype: object 

              precision    recall  f1-score   support

hrudayashiva       1.00      0.67      0.80         3
ravibeligeri       0.67      1.00      0.80         2
  somashaker       1.00      1.00      1.00         3

 avg / total       0.92      0.88      0.88         8

Enter the string
ಮಾಂಗಲ್ಯಂ ತಂತುನಾನೇನ ಮಮಜೀವನ ಹೇತುನಾ … ಎಂಬ ಮಂತ್ರ
{'hrudayashiva'}
